In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
import json

# Funções

In [ ]:
def tratativa(X):  
    '''
    Entrada: series
    X_norm 
        - retira acentos das palavras 
 
    list comprehension
        - remove palavras com hifen
        - remove "."
        - remove palavra com menos de 2 caracter
        - remove palavras com letras maisculas (nomes proprios)
        - remove palavras com caracteres especiais utilizando regex
        - retira palavras escolhidas
        
    return: 2 listas, 1 de palavras sem acentos e outra normal
    '''

    
    #verificacao do regex para caracteres especiais
    string_check = re.compile("[ƒµ‚†„\ˆ¥÷œ‡þ»+'-@_!#$%^&*()<>?/\|}{~:ºª°º¹²³£¢¬]")

    word_list = [x for x in X
                if  '-' not in x 
                and string_check.search(x) == None
                #and len(x) > 2 
                and '.' not in x 
                and x.lower() == x
                ]

    
    word_list_clean = [x for x in word_list if x not in palavroes and x not in norm_palavroes]
        
    
    word_list_clean = [x for x in word_list_clean 
                         if ''.join(
                                 [letter for letter in x 
                                      if not letter.isdigit()]
                                 )
                                ]
    
    word_list_pt = [x for x in word_list_clean if x not in only_en]
    
    # removendo acentos
    y = pd.Series(word_list_pt)
    X_norm = y.apply(lambda row: strip_accents(row))
    
    return X_norm.to_list(), word_list_pt

In [ ]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

# IMPORTANDO DICIONARIO PORTUGUES

In [ ]:
PATH = 'Data/dicionario/lexporbr_alfa_excel.xlsx'
lexico = pd.read_excel(PATH)
lexico.head()

In [ ]:
lexico[lexico.cat_gram == 'ver'].sample(10)

# EDA 

In [ ]:
lexico.ortografia.duplicated().sum()

In [ ]:
lexico.ortografia.isna().sum()

# Iniciando Tratativas

### NULOS E DUPLICADAS

In [ ]:
lexico = lexico.dropna(subset=['ortografia'])

In [ ]:
lexico = lexico.drop_duplicates(subset= 'ortografia', keep='first')

-------

## QUANTIDADE DE LETRAS

#### cria dataframe com todas as palavras maiores que 2 caracteres que não sejam numericos

In [ ]:
lexico_filtered = lexico[(lexico['nb_letras'] > 2) & (lexico.cat_gram != 'num')]

##### cria dataframe com todas as palavras de 2 caracteres que não sejam numericos para futuruas tratativas diferenciadas

In [ ]:
lexico_filtered_2_letras = lexico[(lexico['nb_letras'] == 2) & (lexico.cat_gram != 'num')]

In [ ]:
lexico_filtered_2_letras = lexico_filtered_2_letras.sort_values(by='freq_orto/M', ascending=False)

In [ ]:
lexico_filtered_2_letras.shape

In [ ]:
lexico_filtered_2_letras.head()['ortografia']

-------

# LISTA PALAVRAS EM INGLES

#####  lendo dicionario lexico em ingles

In [ ]:
#http://crr.ugent.be/programs-data/subtitle-frequencies
lexico_en = pd.read_excel(r'Data\dicionario\SUBTLEXusfrequencyabove1.xls')
lexico_en.head()

In [ ]:
lexico_en.shape

##### transformando em lista

In [ ]:
english_words = lexico_en.Word.to_list()

##### ordenando pelas mais frequentes (as palavras em pt br ficarão por ultimo, na 6077 colocação) - leitura visual

In [ ]:
br_en = lexico[lexico.ortografia.isin(english_words)].sort_values(by='freq_orto/M').ortografia.to_list()

In [ ]:
br_en[0:20]

##### apos verificação manual, notei que essas palavras são brasileiras (ou frequentemente utilizadas no brasil) 

In [ ]:
list_br ='''exotica, formulas, scan, magazine, erotica, level, karaoke, guacamole, hospital, bug, bugs, cadaver, câncer, videocassete, vina, vodka, trial, loop, loops, protons, pet, fã, delete, kilo, kilos, cameras, camera, supernatural, short, use, megabytes, polyester, tsunami, médico, live, pátio, inglês, vírus, álbum, mimosa, língua, bio, zap, premium, aliens, alien, pet, pets, hétero, áudio, nerds, nerd, panda, pandas, ninja, ninjas, revise, desktop, anime, cloaca, insular, retro, revolver, indie, neural, macular, medicina, multi, multicultural, auras, aura, container, containers, whisky, whiskies, torso, tiara, ultimo, tutu, local, radio, debutante, flamenco, boxer  , box, combine, cacam baste, basta, barmen, ancora, abates, impostor, iguana, zona, senhorita, led, megawatt, familia, arenas, arena, avance, melodrama, network, poodle, script, semi, juris, jambo, ignore, hacker, gurus, cameraman, avatar, motocross, moto, snowboard, placebo, hobbies, hobby, colas, celestial, celeste, cardiovascular, interface, interfaces, cola, elixir, elixires, vulva, tech, taverna, superego, paranormal, tavernas, portifólio, fondue, escudos, clerical, mandala, manda, número, console, jeep, tempera, silos, poster, pixel, pixels, remix, clique, bike, bikes, cliques, samurais, samurai, sonata, banda, bandas, neuroses, peques, rumba, torpedo, tutorial, modular, consular, carte, intima, intimo, intimas, intimos, utublar, tender, sequela, patina, máscara, iguanas, hangar, ferias, edema, descamisados, jumbo, floral, latitude, gourmet, piano, pianos, marrom, marrons, lambada, fax, faxes, surreal, solicitude, critique, date, examine, replay, remove, vendor, mica, pesetas, dolar, tótem, proverbial, fraternal, nasal, intercontinental, paternal, maternal, feedback, crepe, clone, clones, converse, suíte, merengue, ornamental, sapiens, laptop, lustre, ballet, force, cowboy, cetera, bodes, batom, batons, muleta, tango, pulsar, plebe, mambo, pata, garagem, torque, zoom, subverter, facial, larva, margarita, vascular, servo, virtuoso, presto, predecessor, mole, moles, cache, decimal, declare, capô, temperamental, piranha, piranhas, flamingos, abdominal, cabanas, dons, egos, vertebral, posteriori, recuse, diocese, triangular, radial, pólo, mamas, vesceral, eras, manifestos, manifesto, novena, neutrino, colossal, zebras, zebra, doer, ketchup, mini, posses, posse, vagina, menstrual, menstrua, complete, internet, perpendicular, reserve, marque, libido, tacos, taco, magma, umbilical, burros, jingles, provincial, provincia, shop, sensor, poses, fosse, gospel, causa, hematomas, pampas, sbsolve, chef, chefs, exportar, picante, media, aficionados, vans, van, gringo, infame, ocular, salvos, resorte,transistor, tribal, pedestal, anorexia, cone, cones, drinks, drink, pastoral, portal, fórmula, servos, longitudinal, carnal, frame, chassi, visor, limbo, mucosa, aromas, aroma, renal, volts, fetal, homo, concha, genital, spray, hormonal, anal, dorsal, rifle, paparazzi, menu, casco, caviar, manias, salve, fado, lira, haste, logo, logos, expert, loco, loca, compressor, trailer, longitude, sina, imbórglio, grata, altitudes, orbital, rim, skate, champagne, detector, prepare, angular, fractal, executor, modelo, tutor, tapas, video, papas, papa, nuances, manicure, gringos, feudal, você, lava, rea, televisor, mosquitos, mosquito, vocês, feudais, lavas, machismo, trombone, tequila, revive, soda, ardor, enigmas, enigma, sushi, molar, rata, rato, precede, cover, dental, baby, bolos, note, diagonal, salsa, capacitor, odor, secular, pedal, cia, apologia, realize'''
list_br = list_br.split(', ')

In [ ]:
len(list_br)

##### Apos a 6077 palavra, a maioria das palavras estão em portugues, então criei uma lista pra retirar as de ingles entre elas

In [ ]:
list_en = '''hip, flat, club, jet, must, killer, blazer, it, est, kitsch, heavy, new, royalties, remake, you, rockets, slides, he, business, ops, thriller, notes, made, is, hall, es, by, overnight, country, playoffs, franchising, du, th, cr, soft, food, free, drag, out, soccer, at, cap, star, big, flash, rpm, point, notes, seller, best, zapping, ita, ad, off, qua, quas, es, made, is, mote, line, con, grid, besides, look, pool, s, ex, prime, ohms, ram, game, rg, fr, op, cc, ante, on, to, per, et, en, light, y, sex, n, bi, over, of, di, cm, mi, t, in, kg, iii, te, d, 0'''
list_en = list_en.split(', ')
list_en = pd.Series(list_en).apply(lambda row: strip_accents(row))
list_en = list_en.to_list()

In [ ]:
len(list_en)

#### verifica se as primeiras 6077 palavras da lista de palavras "em ingles" não estão na lista de palavras em br e as proxima ESTÃO  lista de palavras ingles

In [ ]:
#retirando as br da lista de ingles
only_en = [x for x in br_en[0:6077] if x not in list_br] + [y for y in br_en[6077:] if y in list_en]

In [ ]:
only_en[0:50]

--------

# PALAVRÕES

#### lendo o ARQUIVO DE PALAVROES 


In [ ]:
#lendo o ARQUIVO DE PALAVROES 
f = open('Data/palavroes.txt') 
palavroes = f.readlines()
f.close()

In [ ]:
palavroes

#### Tratando palavrões

In [ ]:
palavroes = pd.Series(palavroes).apply(lambda x: x.lower().replace('\n ', '').replace(' ', ''))
norm_palavroes = pd.Series(palavroes).apply(lambda row: strip_accents(row))

--------

# CHAMADA DAS FUNÇÕES

In [ ]:
normalized_word_list, word_list= tratativa(lexico_filtered['ortografia'])

In [ ]:
normalized_word_list_2_letras, word_list_2_letras= tratativa(lexico_filtered_2_letras['ortografia'])

In [ ]:
normalized_word_list_2_letras[0:32]

In [ ]:
'jet' in word_list

# Adicionando palavras com 2 letras na lista de palavras totais (normalizadas e nao normalizadas)

In [ ]:
#apos feito uma inspeção visual, foi defido que das 478 palavras de 2 letras, apenas as 32 primeiras sao relevantes. 
#Mesmo com alguns lixos (lista retirar) e de todass as 478 palavras, algumas ainda seriam relevantes (lista inserir e norm_inserir)
retirar = ['ao', 'km', 'ii', 'mg']
inserir = ['lá', 'fé','tú', 'dê', 'se', 'nó', 'dj', 'br', 'pá', 'dó', 'ok','oi', 'nú']
norm_inserir = ['la', 'fe','tu', 'de', 'se', 'no', 'dj', 'br', 'pa', 'do', 'ok','oi', 'nu']


# as 32 primeiras palavras da lista estão ok
#retirnando as palavras normalizadas escolhidas dentre as 32 primeiras da lista de palavras com 2 letras
normalized_word_list_2_letras = [x for x in normalized_word_list_2_letras[0:32] if x not in retirar] 

#inserindo as palavras normalizadas escolhidas
normalized_word_list_2_letras = normalized_word_list_2_letras + norm_inserir

#somando com a lista de palavras normalizadas todais
normalized_word_list = normalized_word_list + normalized_word_list_2_letras


#repetindo o processo para lista de palavras nao normalizadas
#retirnando as palavras escolhidas dentre as 32 primeiras da lista de palavras com 2 letras dentre as 32 primeiras
word_list_2_letras = [x for x in word_list_2_letras[0:32] if x not in retirar] 

#inserindo as palavras escolhidas
word_list_2_letras = word_list_2_letras + norm_inserir

#somando com a lista de palavras
word_list = word_list + word_list_2_letras

In [ ]:
len(word_list)

# CRIANDO O JSON

In [ ]:
import json

i = 0
dic_aux = {}
list_aux = []

for word, norm_word in zip(word_list, normalized_word_list):
    dic_aux = {
        'id': i,
        'word':word,
        'normalized': norm_word
    }    
    list_aux.append(dic_aux)
    i = i+1

## INSERINDO LISTA DE NUMERAIS DIRETO NO JSON

In [ ]:
numerais = 'um, dois, três, quatro, cinco, seis, sete, oito, nove, dez, onze, doze, treze, quatorze, quinze, dezesseis, dezessete, dezoito, dezenove, vinte, trinta, quarenta, cinquenta, sessenta, setenta, oitenta, noventa, cem, duzentos, trezentos, quatrocentos, quinhentos, seiscentos, setecentos, oitocentos, novecentos, mil'
numerais = numerais.split(', ')
norm_numerais = pd.Series(numerais).apply(lambda row: strip_accents(row))


list_aux2 = []
for i in range (len(list_aux), len(list_aux) +  len(numerais)):
    dic_aux = {
        'id': i,
        'word': numerais[i - len(list_aux)],
        'normalized' : norm_numerais.to_list()[i - len(list_aux)]        
    }
    
    list_aux2.append(dic_aux)

In [ ]:
list_aux = list_aux + list_aux2

In [ ]:
len(list_aux)

## SALVANDO O JSON

In [ ]:
#salvando o arquivo json
#with open('Data/words.json', 'w') as f:
#    json.dump(list_aux, f)

----

# RESULTADOS:

In [ ]:
import json
import pandas as pd

#### LENDO O JSON

In [ ]:
#lendo o JSON 
f = open('Data/words.json') 
words_file = json.load(f)
f.close()

In [ ]:
word_list = []
normalized_word_list =[]

for dic in words_file:
    word_list.append(dic['word'])
    normalized_word_list.append(dic['normalized'])

In [ ]:
dic = {'word': word_list,
      'norm_word': normalized_word_list}

In [ ]:
words_df = pd.DataFrame(dic, columns =['word', 'norm_word'])

In [ ]:
words_df

------------

# Adicionais

####  - CONJULGANDO VERBOS

In [ ]:
PATH = 'Data/dicionario/lexporbr_alfa_excel.xlsx'
lexico = pd.read_excel(PATH)
lexico.head()

#### tratando verbos irregulares

In [ ]:
lexico

In [ ]:
#html com lista de verbos irregulares de https://www.conjugacao.com.br/verbos-irregulares/
irregular_verbs_html = '''<div class="wrapper">
    <ul><li><a href="/verbo-abster/" title="Verbo abster">abster</a></li><li><a href="/verbo-acudir/" title="Verbo acudir">acudir</a></li><li><a href="/verbo-adequar/" title="Verbo adequar">adequar</a></li><li><a href="/verbo-aderir/" title="Verbo aderir">aderir</a></li><li><a href="/verbo-adjazer/" title="Verbo adjazer">adjazer</a></li><li><a href="/verbo-advertir/" title="Verbo advertir">advertir</a></li><li><a href="/verbo-advir/" title="Verbo advir">advir</a></li><li><a href="/verbo-afazer/" title="Verbo afazer">afazer</a></li><li><a href="/verbo-aferir/" title="Verbo aferir">aferir</a></li><li><a href="/verbo-agredir/" title="Verbo agredir">agredir</a></li><li><a href="/verbo-ansiar/" title="Verbo ansiar">ansiar</a></li><li><a href="/verbo-antedar/" title="Verbo antedar">antedar</a></li><li><a href="/verbo-antepor/" title="Verbo antepor">antepor</a></li><li><a href="/verbo-antever/" title="Verbo antever">antever</a></li><li><a href="/verbo-apor/" title="Verbo apor">apor</a></li><li><a href="/verbo-aprazer/" title="Verbo aprazer">aprazer</a></li><li><a href="/verbo-apropinquar/" title="Verbo apropinquar">apropinquar</a></li><li><a href="/verbo-aspergir/" title="Verbo aspergir">aspergir</a></li><li><a href="/verbo-assentir/" title="Verbo assentir">assentir</a></li><li><a href="/verbo-ater/" title="Verbo ater">ater</a></li><li><a href="/verbo-atrair/" title="Verbo atrair">atrair</a></li><li><a href="/verbo-atribuir/" title="Verbo atribuir">atribuir</a></li><li><a href="/verbo-auferir/" title="Verbo auferir">auferir</a></li><li><a href="/verbo-autodestruir/" title="Verbo autodestruir">autodestruir</a></li><li><a href="/verbo-avir/" title="Verbo avir">avir</a></li><li><a href="/verbo-bem-dizer/" title="Verbo bem-dizer">bem-dizer</a></li><li><a href="/verbo-bem-fazer/" title="Verbo bem-fazer">bem-fazer</a></li><li><a href="/verbo-bem-querer/" title="Verbo bem-querer">bem-querer</a></li><li><a href="/verbo-bendizer/" title="Verbo bendizer">bendizer</a></li><li><a href="/verbo-benfazer/" title="Verbo benfazer">benfazer</a></li><li><a href="/verbo-benquerer/" title="Verbo benquerer">benquerer</a></li><li><a href="/verbo-buir/" title="Verbo buir">buir</a></li><li><a href="/verbo-bulir/" title="Verbo bulir">bulir</a></li><li><a href="/verbo-caber/" title="Verbo caber">caber</a></li><li><a href="/verbo-cair/" title="Verbo cair">cair</a></li><li><a href="/verbo-cerzir/" title="Verbo cerzir">cerzir</a></li><li><a href="/verbo-circumpor/" title="Verbo circumpor">circumpor</a></li><li><a href="/verbo-circunver/" title="Verbo circunver">circunver</a></li><li><a href="/verbo-cobrir/" title="Verbo cobrir">cobrir</a></li><li><a href="/verbo-compelir/" title="Verbo compelir">compelir</a></li><li><a href="/verbo-competir/" title="Verbo competir">competir</a></li><li><a href="/verbo-compor/" title="Verbo compor">compor</a></li><li><a href="/verbo-comprazer/" title="Verbo comprazer">comprazer</a></li><li><a href="/verbo-concernir/" title="Verbo concernir">concernir</a></li><li><a href="/verbo-concluir/" title="Verbo concluir">concluir</a></li><li><a href="/verbo-condizer/" title="Verbo condizer">condizer</a></li><li><a href="/verbo-conferir/" title="Verbo conferir">conferir</a></li><li><a href="/verbo-confugir/" title="Verbo confugir">confugir</a></li><li><a href="/verbo-conseguir/" title="Verbo conseguir">conseguir</a></li><li><a href="/verbo-consentir/" title="Verbo consentir">consentir</a></li><li><a href="/verbo-construir/" title="Verbo construir">construir</a></li><li><a href="/verbo-consumir/" title="Verbo consumir">consumir</a></li><li><a href="/verbo-conter/" title="Verbo conter">conter</a></li><li><a href="/verbo-contradizer/" title="Verbo contradizer">contradizer</a></li><li><a href="/verbo-contrafazer/" title="Verbo contrafazer">contrafazer</a></li><li><a href="/verbo-contrair/" title="Verbo contrair">contrair</a></li><li><a href="/verbo-contrapor/" title="Verbo contrapor">contrapor</a></li><li><a href="/verbo-contrapropor/" title="Verbo contrapropor">contrapropor</a></li><li><a href="/verbo-contravir/" title="Verbo contravir">contravir</a></li><li><a href="/verbo-convergir/" title="Verbo convergir">convergir</a></li><li><a href="/verbo-convir/" title="Verbo convir">convir</a></li><li><a href="/verbo-crer/" title="Verbo crer">crer</a></li><li><a href="/verbo-cuspir/" title="Verbo cuspir">cuspir</a></li><li><a href="/verbo-dar/" title="Verbo dar">dar</a></li><li><a href="/verbo-decompor/" title="Verbo decompor">decompor</a></li><li><a href="/verbo-deferir/" title="Verbo deferir">deferir</a></li><li><a href="/verbo-delinquir/" title="Verbo delinquir">delinquir</a></li><li><a href="/verbo-denegrir/" title="Verbo denegrir">denegrir</a></li><li><a href="/verbo-depor/" title="Verbo depor">depor</a></li><li><a href="/verbo-desafazer/" title="Verbo desafazer">desafazer</a></li><li><a href="/verbo-desaguar/" title="Verbo desaguar">desaguar</a></li><li><a href="/verbo-desapor/" title="Verbo desapor">desapor</a></li><li><a href="/verbo-desaprazer/" title="Verbo desaprazer">desaprazer</a></li><li><a href="/verbo-desavir/" title="Verbo desavir">desavir</a></li><li><a href="/verbo-descaber/" title="Verbo descaber">descaber</a></li><li><a href="/verbo-descobrir/" title="Verbo descobrir">descobrir</a></li><li><a href="/verbo-descompor/" title="Verbo descompor">descompor</a></li><li><a href="/verbo-descomprazer/" title="Verbo descomprazer">descomprazer</a></li><li><a href="/verbo-desconstruir/" title="Verbo desconstruir">desconstruir</a></li><li><a href="/verbo-desconvir/" title="Verbo desconvir">desconvir</a></li><li><a href="/verbo-descrer/" title="Verbo descrer">descrer</a></li><li><a href="/verbo-desdar/" title="Verbo desdar">desdar</a></li><li><a href="/verbo-desdizer/" title="Verbo desdizer">desdizer</a></li><li><a href="/verbo-desimpedir/" title="Verbo desimpedir">desimpedir</a></li><li><a href="/verbo-desimpor/" title="Verbo desimpor">desimpor</a></li><li><a href="/verbo-deslinguar/" title="Verbo deslinguar">deslinguar</a></li><li><a href="/verbo-desmedir/" title="Verbo desmedir">desmedir</a></li><li><a href="/verbo-desmentir/" title="Verbo desmentir">desmentir</a></li><li><a href="/verbo-desmobiliar/" title="Verbo desmobiliar">desmobiliar</a></li><li><a href="/verbo-despedir/" title="Verbo despedir">despedir</a></li><li><a href="/verbo-despir/" title="Verbo despir">despir</a></li><li><a href="/verbo-despolir/" title="Verbo despolir">despolir</a></li><li><a href="/verbo-despor/" title="Verbo despor">despor</a></li><li><a href="/verbo-desprazer/" title="Verbo desprazer">desprazer</a></li><li><a href="/verbo-desprecaver/" title="Verbo desprecaver">desprecaver</a></li><li><a href="/verbo-desprover/" title="Verbo desprover">desprover</a></li><li><a href="/verbo-desquerer/" title="Verbo desquerer">desquerer</a></li><li><a href="/verbo-dessaber/" title="Verbo dessaber">dessaber</a></li><li><a href="/verbo-destruir/" title="Verbo destruir">destruir</a></li><li><a href="/verbo-desvaler/" title="Verbo desvaler">desvaler</a></li><li><a href="/verbo-desver/" title="Verbo desver">desver</a></li><li><a href="/verbo-deter/" title="Verbo deter">deter</a></li><li><a href="/verbo-devir/" title="Verbo devir">devir</a></li><li><a href="/verbo-diferir/" title="Verbo diferir">diferir</a></li><li><a href="/verbo-digerir/" title="Verbo digerir">digerir</a></li><li><a href="/verbo-disferir/" title="Verbo disferir">disferir</a></li><li><a href="/verbo-disperder/" title="Verbo disperder">disperder</a></li><li><a href="/verbo-dispor/" title="Verbo dispor">dispor</a></li><li><a href="/verbo-distrair/" title="Verbo distrair">distrair</a></li><li><a href="/verbo-divergir/" title="Verbo divergir">divergir</a></li><li><a href="/verbo-divertir/" title="Verbo divertir">divertir</a></li><li><a href="/verbo-dizer/" title="Verbo dizer">dizer</a></li><li><a href="/verbo-dormir/" title="Verbo dormir">dormir</a></li><li><a href="/verbo-embair/" title="Verbo embair">embair</a></li><li><a href="/verbo-emergir/" title="Verbo emergir">emergir</a></li><li><a href="/verbo-encobrir/" title="Verbo encobrir">encobrir</a></li><li><a href="/verbo-engolir/" title="Verbo engolir">engolir</a></li><li><a href="/verbo-entredizer/" title="Verbo entredizer">entredizer</a></li><li><a href="/verbo-entrefazer/" title="Verbo entrefazer">entrefazer</a></li><li><a href="/verbo-entreouvir/" title="Verbo entreouvir">entreouvir</a></li><li><a href="/verbo-entrepor/" title="Verbo entrepor">entrepor</a></li><li><a href="/verbo-entrequerer/" title="Verbo entrequerer">entrequerer</a></li><li><a href="/verbo-entrever/" title="Verbo entrever">entrever</a></li><li><a href="/verbo-entrevir/" title="Verbo entrevir">entrevir</a></li><li><a href="/verbo-entupir/" title="Verbo entupir">entupir</a></li><li><a href="/verbo-enxaguar/" title="Verbo enxaguar">enxaguar</a></li><li><a href="/verbo-enxerir/" title="Verbo enxerir">enxerir</a></li><li><a href="/verbo-equivaler/" title="Verbo equivaler">equivaler</a></li><li><a href="/verbo-escapulir/" title="Verbo escapulir">escapulir</a></li><li><a href="/verbo-esfazer/" title="Verbo esfazer">esfazer</a></li><li><a href="/verbo-estar/" title="Verbo estar">estar</a></li><li><a href="/verbo-estrear/" title="Verbo estrear">estrear</a></li><li><a href="/verbo-esvair/" title="Verbo esvair">esvair</a></li><li><a href="/verbo-expedir/" title="Verbo expedir">expedir</a></li><li><a href="/verbo-expelir/" title="Verbo expelir">expelir</a></li><li><a href="/verbo-expor/" title="Verbo expor">expor</a></li><li><a href="/verbo-extrapor/" title="Verbo extrapor">extrapor</a></li><li><a href="/verbo-fazer/" title="Verbo fazer">fazer</a></li><li><a href="/verbo-ferir/" title="Verbo ferir">ferir</a></li><li><a href="/verbo-flectir/" title="Verbo flectir">flectir</a></li><li><a href="/verbo-fletir/" title="Verbo fletir">fletir</a></li><li><a href="/verbo-fotocompor/" title="Verbo fotocompor">fotocompor</a></li><li><a href="/verbo-fraguar/" title="Verbo fraguar">fraguar</a></li><li><a href="/verbo-frigir/" title="Verbo frigir">frigir</a></li><li><a href="/verbo-fugir/" title="Verbo fugir">fugir</a></li><li><a href="/verbo-gelifazer/" title="Verbo gelifazer">gelifazer</a></li><li><a href="/verbo-gerir/" title="Verbo gerir">gerir</a></li><li><a href="/verbo-haver/" title="Verbo haver">haver</a></li><li><a href="/verbo-idear/" title="Verbo idear">idear</a></li><li><a href="/verbo-imergir/" title="Verbo imergir">imergir</a></li><li><a href="/verbo-impedir/" title="Verbo impedir">impedir</a></li><li><a href="/verbo-impelir/" title="Verbo impelir">impelir</a></li><li><a href="/verbo-impor/" title="Verbo impor">impor</a></li><li><a href="/verbo-incendiar/" title="Verbo incendiar">incendiar</a></li><li><a href="/verbo-indeferir/" title="Verbo indeferir">indeferir</a></li><li><a href="/verbo-indispor/" title="Verbo indispor">indispor</a></li><li><a href="/verbo-inferir/" title="Verbo inferir">inferir</a></li><li><a href="/verbo-influir/" title="Verbo influir">influir</a></li><li><a href="/verbo-ingerir/" title="Verbo ingerir">ingerir</a></li><li><a href="/verbo-insatisfazer/" title="Verbo insatisfazer">insatisfazer</a></li><li><a href="/verbo-inserir/" title="Verbo inserir">inserir</a></li><li><a href="/verbo-interdizer/" title="Verbo interdizer">interdizer</a></li><li><a href="/verbo-intermediar/" title="Verbo intermediar">intermediar</a></li><li><a href="/verbo-interpor/" title="Verbo interpor">interpor</a></li><li><a href="/verbo-interver/" title="Verbo interver">interver</a></li><li><a href="/verbo-intervir/" title="Verbo intervir">intervir</a></li><li><a href="/verbo-investir/" title="Verbo investir">investir</a></li><li><a href="/verbo-ir/" title="Verbo ir">ir</a></li><li><a href="/verbo-jazer/" title="Verbo jazer">jazer</a></li><li><a href="/verbo-justapor/" title="Verbo justapor">justapor</a></li><li><a href="/verbo-ler/" title="Verbo ler">ler</a></li><li><a href="/verbo-liquefazer/" title="Verbo liquefazer">liquefazer</a></li><li><a href="/verbo-maisquerer/" title="Verbo maisquerer">maisquerer</a></li><li><a href="/verbo-maldispor/" title="Verbo maldispor">maldispor</a></li><li><a href="/verbo-maldizer/" title="Verbo maldizer">maldizer</a></li><li><a href="/verbo-malfazer/" title="Verbo malfazer">malfazer</a></li><li><a href="/verbo-malinguar/" title="Verbo malinguar">malinguar</a></li><li><a href="/verbo-malparir/" title="Verbo malparir">malparir</a></li><li><a href="/verbo-malquerer/" title="Verbo malquerer">malquerer</a></li><li><a href="/verbo-manter/" title="Verbo manter">manter</a></li><li><a href="/verbo-mediar/" title="Verbo mediar">mediar</a></li><li><a href="/verbo-medir/" title="Verbo medir">medir</a></li><li><a href="/verbo-mentir/" title="Verbo mentir">mentir</a></li><li><a href="/verbo-minguar/" title="Verbo minguar">minguar</a></li><li><a href="/verbo-obter/" title="Verbo obter">obter</a></li><li><a href="/verbo-obvir/" title="Verbo obvir">obvir</a></li><li><a href="/verbo-odiar/" title="Verbo odiar">odiar</a></li><li><a href="/verbo-opor/" title="Verbo opor">opor</a></li><li><a href="/verbo-ouvir/" title="Verbo ouvir">ouvir</a></li><li><a href="/verbo-parir/" title="Verbo parir">parir</a></li><li><a href="/verbo-pedir/" title="Verbo pedir">pedir</a></li><li><a href="/verbo-perder/" title="Verbo perder">perder</a></li><li><a href="/verbo-perfazer/" title="Verbo perfazer">perfazer</a></li><li><a href="/verbo-perseguir/" title="Verbo perseguir">perseguir</a></li><li><a href="/verbo-persentir/" title="Verbo persentir">persentir</a></li><li><a href="/verbo-pleitear/" title="Verbo pleitear">pleitear</a></li><li><a href="/verbo-poder/" title="Verbo poder">poder</a></li><li><a href="/verbo-poer/" title="Verbo poer">poer</a></li><li><a href="/verbo-polir/" title="Verbo polir">polir</a></li><li><a href="/verbo-pospor/" title="Verbo pospor">pospor</a></li><li><a href="/verbo-por/" title="Verbo pôr">pôr</a></li><li><a href="/verbo-prazer/" title="Verbo prazer">prazer</a></li><li><a href="/verbo-predispor/" title="Verbo predispor">predispor</a></li><li><a href="/verbo-predizer/" title="Verbo predizer">predizer</a></li><li><a href="/verbo-preferir/" title="Verbo preferir">preferir</a></li><li><a href="/verbo-prepor/" title="Verbo prepor">prepor</a></li><li><a href="/verbo-pressentir/" title="Verbo pressentir">pressentir</a></li><li><a href="/verbo-pressupor/" title="Verbo pressupor">pressupor</a></li><li><a href="/verbo-preterir/" title="Verbo preterir">preterir</a></li><li><a href="/verbo-prevenir/" title="Verbo prevenir">prevenir</a></li><li><a href="/verbo-prever/" title="Verbo prever">prever</a></li><li><a href="/verbo-proferir/" title="Verbo proferir">proferir</a></li><li><a href="/verbo-progredir/" title="Verbo progredir">progredir</a></li><li><a href="/verbo-propor/" title="Verbo propor">propor</a></li><li><a href="/verbo-prosseguir/" title="Verbo prosseguir">prosseguir</a></li><li><a href="/verbo-prossupor/" title="Verbo prossupor">prossupor</a></li><li><a href="/verbo-prover/" title="Verbo prover">prover</a></li><li><a href="/verbo-provir/" title="Verbo provir">provir</a></li><li><a href="/verbo-pruir/" title="Verbo pruir">pruir</a></li><li><a href="/verbo-puir/" title="Verbo puir">puir</a></li><li><a href="/verbo-putrefazer/" title="Verbo putrefazer">putrefazer</a></li><li><a href="/verbo-querer/" title="Verbo querer">querer</a></li><li><a href="/verbo-raer/" title="Verbo raer">raer</a></li><li><a href="/verbo-rarefazer/" title="Verbo rarefazer">rarefazer</a></li><li><a href="/verbo-readequar/" title="Verbo readequar">readequar</a></li><li><a href="/verbo-reaver/" title="Verbo reaver">reaver</a></li><li><a href="/verbo-reavir/" title="Verbo reavir">reavir</a></li><li><a href="/verbo-recobrir/" title="Verbo recobrir">recobrir</a></li><li><a href="/verbo-recompor/" title="Verbo recompor">recompor</a></li><li><a href="/verbo-reconvir/" title="Verbo reconvir">reconvir</a></li><li><a href="/verbo-redar/" title="Verbo redar">redar</a></li><li><a href="/verbo-redispor/" title="Verbo redispor">redispor</a></li><li><a href="/verbo-redizer/" title="Verbo redizer">redizer</a></li><li><a href="/verbo-reexpedir/" title="Verbo reexpedir">reexpedir</a></li><li><a href="/verbo-reexpor/" title="Verbo reexpor">reexpor</a></li><li><a href="/verbo-refazer/" title="Verbo refazer">refazer</a></li><li><a href="/verbo-referir/" title="Verbo referir">referir</a></li><li><a href="/verbo-refletir/" title="Verbo refletir">refletir</a></li><li><a href="/verbo-refugir/" title="Verbo refugir">refugir</a></li><li><a href="/verbo-regredir/" title="Verbo regredir">regredir</a></li><li><a href="/verbo-reimpor/" title="Verbo reimpor">reimpor</a></li><li><a href="/verbo-reindispor/" title="Verbo reindispor">reindispor</a></li><li><a href="/verbo-reinserir/" title="Verbo reinserir">reinserir</a></li><li><a href="/verbo-reler/" title="Verbo reler">reler</a></li><li><a href="/verbo-remediar/" title="Verbo remediar">remediar</a></li><li><a href="/verbo-remedir/" title="Verbo remedir">remedir</a></li><li><a href="/verbo-reobter/" title="Verbo reobter">reobter</a></li><li><a href="/verbo-reouvir/" title="Verbo reouvir">reouvir</a></li><li><a href="/verbo-repedir/" title="Verbo repedir">repedir</a></li><li><a href="/verbo-repelir/" title="Verbo repelir">repelir</a></li><li><a href="/verbo-repetir/" title="Verbo repetir">repetir</a></li><li><a href="/verbo-repor/" title="Verbo repor">repor</a></li><li><a href="/verbo-repropor/" title="Verbo repropor">repropor</a></li><li><a href="/verbo-requerer/" title="Verbo requerer">requerer</a></li><li><a href="/verbo-resfolegar/" title="Verbo resfolegar">resfolegar</a></li><li><a href="/verbo-ressentir/" title="Verbo ressentir">ressentir</a></li><li><a href="/verbo-reter/" title="Verbo reter">reter</a></li><li><a href="/verbo-retrair/" title="Verbo retrair">retrair</a></li><li><a href="/verbo-retranspor/" title="Verbo retranspor">retranspor</a></li><li><a href="/verbo-rever/" title="Verbo rever">rever</a></li><li><a href="/verbo-revestir/" title="Verbo revestir">revestir</a></li><li><a href="/verbo-revir/" title="Verbo revir">revir</a></li><li><a href="/verbo-rir/" title="Verbo rir">rir</a></li><li><a href="/verbo-ruir/" title="Verbo ruir">ruir</a></li><li><a href="/verbo-saber/" title="Verbo saber">saber</a></li><li><a href="/verbo-sacudir/" title="Verbo sacudir">sacudir</a></li><li><a href="/verbo-sair/" title="Verbo sair">sair</a></li><li><a href="/verbo-santiguar/" title="Verbo santiguar">santiguar</a></li><li><a href="/verbo-satisfazer/" title="Verbo satisfazer">satisfazer</a></li><li><a href="/verbo-seguir/" title="Verbo seguir">seguir</a></li><li><a href="/verbo-sentir/" title="Verbo sentir">sentir</a></li><li><a href="/verbo-ser/" title="Verbo ser">ser</a></li><li><a href="/verbo-servir/" title="Verbo servir">servir</a></li><li><a href="/verbo-sobpor/" title="Verbo sobpor">sobpor</a></li><li><a href="/verbo-sobre-expor/" title="Verbo sobre-expor">sobre-expor</a></li><li><a href="/verbo-sobreexpor/" title="Verbo sobreexpor">sobreexpor</a></li><li><a href="/verbo-sobrepor/" title="Verbo sobrepor">sobrepor</a></li><li><a href="/verbo-sobrestar/" title="Verbo sobrestar">sobrestar</a></li><li><a href="/verbo-sobrevir/" title="Verbo sobrevir">sobrevir</a></li><li><a href="/verbo-sorrir/" title="Verbo sorrir">sorrir</a></li><li><a href="/verbo-sortear/" title="Verbo sortear">sortear</a></li><li><a href="/verbo-sortir/" title="Verbo sortir">sortir</a></li><li><a href="/verbo-sotopor/" title="Verbo sotopor">sotopor</a></li><li><a href="/verbo-subir/" title="Verbo subir">subir</a></li><li><a href="/verbo-submergir/" title="Verbo submergir">submergir</a></li><li><a href="/verbo-subpor/" title="Verbo subpor">subpor</a></li><li><a href="/verbo-subsumir/" title="Verbo subsumir">subsumir</a></li><li><a href="/verbo-subtrair/" title="Verbo subtrair">subtrair</a></li><li><a href="/verbo-sugerir/" title="Verbo sugerir">sugerir</a></li><li><a href="/verbo-sumir/" title="Verbo sumir">sumir</a></li><li><a href="/verbo-superexpor/" title="Verbo superexpor">superexpor</a></li><li><a href="/verbo-superimpor/" title="Verbo superimpor">superimpor</a></li><li><a href="/verbo-superpor/" title="Verbo superpor">superpor</a></li><li><a href="/verbo-supor/" title="Verbo supor">supor</a></li><li><a href="/verbo-suster/" title="Verbo suster">suster</a></li><li><a href="/verbo-telever/" title="Verbo telever">telever</a></li><li><a href="/verbo-ter/" title="Verbo ter">ter</a></li><li><a href="/verbo-torrefazer/" title="Verbo torrefazer">torrefazer</a></li><li><a href="/verbo-tossir/" title="Verbo tossir">tossir</a></li><li><a href="/verbo-trair/" title="Verbo trair">trair</a></li><li><a href="/verbo-transfazer/" title="Verbo transfazer">transfazer</a></li><li><a href="/verbo-transferir/" title="Verbo transferir">transferir</a></li><li><a href="/verbo-transfugir/" title="Verbo transfugir">transfugir</a></li><li><a href="/verbo-transgredir/" title="Verbo transgredir">transgredir</a></li><li><a href="/verbo-transpor/" title="Verbo transpor">transpor</a></li><li><a href="/verbo-traspor/" title="Verbo traspor">traspor</a></li><li><a href="/verbo-trazer/" title="Verbo trazer">trazer</a></li><li><a href="/verbo-treler/" title="Verbo treler">treler</a></li><li><a href="/verbo-tresler/" title="Verbo tresler">tresler</a></li><li><a href="/verbo-trespor/" title="Verbo trespor">trespor</a></li><li><a href="/verbo-tumefazer/" title="Verbo tumefazer">tumefazer</a></li><li><a href="/verbo-valer/" title="Verbo valer">valer</a></li><li><a href="/verbo-ver/" title="Verbo ver">ver</a></li><li><a href="/verbo-vestir/" title="Verbo vestir">vestir</a></li><li><a href="/verbo-vir/" title="Verbo vir">vir</a></li>    </ul>
</div>'''

In [ ]:
#extraindo a lista
from bs4 import BeautifulSoup as soup

_div = soup(irregular_verbs_html, 'html.parser').find('div', {'class':'wrapper'})
irregular_verbs = [[i.text for i in b.find_all('li')] for b in _div.find_all('ul')][0]

In [ ]:
irregular_verbs

In [ ]:
irregular_verbs[0:5]

In [ ]:
normal_verbs = words_df[~words_df.word.isin(irregular_verbs)]

In [ ]:
len(normal_verbs)

In [ ]:
normal_verbs

In [ ]:
ganhar - eu ganho, tu ganhas, nos ganhamos 
bater - eu bato, tu bates 
entupir - eu entupo, tu etopes

In [ ]:
verbs['termina em'] = verbs.ortografiaverbo[-2:]  # separa a terminação do verbo 

In [ ]:
conjuga_ar = ['o', 'as', 'a', 'amos', 'ais', 'am']; 
conjuga_er = ['o', 'es', 'e', 'emos', 'eis', 'em']; 
conjuga_ir = ['o', 'es', 'e', 'imos', 'is', 'em']; 

In [ ]:
verbs.ortografia